In [1]:
from pybaseball import statcast
from pybaseball import playerid_lookup
from pybaseball import statcast_pitcher
from pybaseball import statcast_batter

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

In [4]:
data = statcast(start_dt='2021-04-01', end_dt='2021-10-03')

This is a large query, it may take a moment to complete


100%|█████████████████████████████████████████| 186/186 [00:20<00:00,  9.15it/s]


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
2877,FF,2021-10-03,92.3,1.4,6.8,"Smith, Will",596019,519293,field_out,hit_into_play,...,5,0,5,0,5,Infield shift,Strategic,148,0.0,-0.073
3019,SL,2021-10-03,80.6,1.6,6.64,"Smith, Will",596019,519293,None,foul,...,5,0,5,0,5,Infield shift,Strategic,315,0.0,-0.027


In [7]:
players = pd.read_csv("PlayerIDMap.csv")

In [8]:
players = players.set_index('MLBID')

In [9]:
on_2= data[data['on_2b'].notnull()][['on_2b', 'inning_topbot' ,'woba_value', 'woba_denom', 'delta_run_exp', 'delta_home_win_exp']]

In [10]:
x = on_2.groupby(['on_2b']).sum()

In [11]:
x2 = x.sort_values('delta_run_exp', ascending = False)

In [12]:
x3 = x2.join(players)

In [13]:
x3['new_col'] = x3['delta_run_exp']/x3['woba_denom']

In [14]:
x4 = x3[['TEAM','woba_value', 'woba_denom', 'delta_run_exp', 'delta_home_win_exp', 'new_col' ,'PLAYERNAME', 'POS', 'BIRTHDATE']]

In [15]:
x5 = x4[x4['woba_denom']>20] #filter out those who didn't play much

In [16]:
x6 = x5.sort_values('new_col', ascending = False)[['PLAYERNAME','POS', 'new_col']]

In [18]:
x6.head(20)

,PLAYERNAME,POS,new_col
595978,Austin Hedges,C,0.359393
677551,Wander Franco,SS,0.342525
643376,Danny Jansen,C,0.2998
621512,Tomas Nido,C,0.299696
641505,Jonathan Davis,OF,0.275214
666211,Taylor Trammell,OF,0.265609
621311,David Dahl,OF,0.244683
664059,Sam Haggerty,OF,0.236727
670712,Michael Brosseau,2B,0.236353
643289,Mauricio Dubon,SS,0.228259


In [19]:
x4[x4['TEAM']=='TOR'].sort_values('new_col', ascending = False)

,TEAM,woba_value,woba_denom,delta_run_exp,delta_home_win_exp,new_col,PLAYERNAME,POS,BIRTHDATE
670950,TOR,2.0,2,2.444,0.073,1.222,Trevor Richards,P,5/15/1993
643376,TOR,17.75,35,10.493,-0.955,0.2998,Danny Jansen,C,4/15/1995
571875,TOR,10.55,27,5.818,0.195,0.215481,Jacob Lamb,OF,10/9/1990
592332,TOR,4.15,9,1.848,0.078,0.205333,Kevin Gausman,P,1/6/1991
666971,TOR,36.35,94,10.477,-1.043,0.111457,Lourdes Gurriel,OF,10/19/1993
624415,TOR,20.3,53,5.423,-0.575,0.102321,Cavan Biggio,3B,4/11/1995
666182,TOR,62.1,159,12.802,-0.763,0.080516,Bo Bichette,SS,3/5/1998
606192,TOR,35.95,106,7.998,0.045,0.075453,Teoscar Hernandez,OF,10/15/1992
543807,TOR,24.3,63,3.862,-1.788,0.061302,George Springer,OF,9/19/1989
669289,TOR,20.15,58,0.958,-0.082,0.016517,Santiago Espinal,3B,11/13/1994
